# Important Graphs for analysis in the case of ENSE3

In [1]:
# --------------------------------
# Required Library Installation
# --------------------------------

import sys
import subprocess

# List of required Python libraries for the project
required_libraries = [
    'pandas',            # Data processing and manipulation
    'numpy',             # Numerical calculations
    'plotly',            # Interactive visualizations
    'ipywidgets',        # Interactive widgets for user input
    'matplotlib',        # Plotting capabilities 
]

# Function to check and install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing missing library: {lib}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        else:
            print(f"✔ {lib} is already installed.")

# Run the function to install missing libraries
install_missing_libraries(required_libraries)


✔ pandas is already installed.
✔ numpy is already installed.
✔ plotly is already installed.
✔ ipywidgets is already installed.
✔ matplotlib is already installed.


In [2]:
# Cell 2: Importing Modules and Reloading solar_analysis_script
import os
import sys
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import solar_analysis_script as sas
import datetime

# Reload the updated solar_analysis_script module in case of changes
import importlib
importlib.reload(sas)

# Cell 3: Define File Paths and Parameters

# Define file paths
consumption_file = 'Ense3buildingconsumption.csv'
pv_file = 'Ense3buildingPV.csv'

# Define separate date ranges
consumption_start_date = '2023-09-25 00:00:00'
consumption_end_date = '2024-09-25 00:00:00'

# Specify the actual column names in your consumption CSV
consumption_time_col = 'Time'
consumption_value_col = 'Consumption (kWh)'

# Number of solar panels (initial default value; can be adjusted via widget)
num_panels = 100  # Example value; adjust as needed

# Cell 4: Load and Merge Data

# Load and Merge Data

try:
    merged_data = sas.load_data(
        consumption_file, 
        pv_file, 
        consumption_start_date, 
        consumption_end_date,
        consumption_sep=';', 
        consumption_time_col=consumption_time_col, 
        consumption_value_col=consumption_value_col,
        pv_sep=','
    )
    print("Data successfully loaded and merged after cleaning.")
except Exception as e:
    print(f"An error occurred while loading data: {e}")

# Ensure index is a DatetimeIndex
merged_data.index = pd.to_datetime(merged_data.index)

# Cell 5: Define System Parameters

# Define System Parameters
battery_capacity_per_unit = 57  # Example: 57 kWh per battery
converter_efficiency = 0.9    # 90% converter efficiency
initial_soc = 50              # 50% initial state of charge on the first day
battery_soc_tracking = {}     # Initialize empty dict to track SOC for each day


INFO:solar_analysis_script:Loading energy consumption data from file: Ense3buildingconsumption.csv
INFO:solar_analysis_script:Energy consumption data successfully loaded and filtered.
INFO:solar_analysis_script:Loading PV production data from file: Ense3buildingPV.csv
INFO:solar_analysis_script:'Time_End' column dropped from PV production data.
INFO:solar_analysis_script:PV production data successfully loaded, converted to kWh, and processed.
INFO:solar_analysis_script:Extending PV production data to cover 2023-09-25 00:00:00 to 2024-09-25 00:00:00.
INFO:solar_analysis_script:Original PV data duration: 364 days 23:00:00 (8760 hours)
INFO:solar_analysis_script:Total duration to cover: 366 days 00:00:00 (8784 hours)
INFO:solar_analysis_script:Repeating PV data 2 times to cover the desired period.
INFO:solar_analysis_script:PV production data extended from 2023-09-25 00:00:00 to 2024-09-24 23:00:00.
INFO:solar_analysis_script:Total PV production records after extension: 8784
INFO:solar_an

Data successfully loaded and merged after cleaning.


In [3]:
import numpy as np
import plotly.graph_objects as go

# ---------------------------
# City-Level Annual SSR Analysis (PV Penetration Ratio)
# ---------------------------

# 1) Compute total PV production over the year
total_pv_power_production = merged_data['PV_Production_kWh'].sum()

# 2) Compute average production over the year (e.g., 8760 hours)
average_pv_power_production = total_pv_power_production / 8760

# 3) Calculate total annual energy demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# 4) Compute average demand in kW (again, dividing by 8760)
average_demand = total_energy_demand / 8760

# 5) Define the range of # of panels
num_panels_range = np.arange(0, 10000, 100)

# Initialize lists
ssr_values = []
pv_penetration_ratio = []

for num_panels in num_panels_range:
    # installed_capacity in kW = (# panels) * (avg PV power from 1 'panel'?)
    #  => You may need to define exactly how 1 panel relates to 'PV_Production_kWh' 
    installed_capacity = num_panels * average_pv_power_production
    
    ratio = (installed_capacity / average_demand) if average_demand != 0 else 0
    pv_penetration_ratio.append(ratio)

    total_energy_supplied_pv = 0.0
    
    # Loop over each hour in merged_data
    for timestamp in merged_data.index:
        # consumption
        consumption = merged_data['Consumption_kWh'].loc[timestamp]
        # PV production that hour
        # (Double-check the logic: 'merged_data["PV_Production_kWh"]' might already be
        #  for 1 "kW" or 1 "panel"? Adjust accordingly.)
        pv_production = merged_data['PV_Production_kWh'].loc[timestamp] * num_panels * average_pv_power_production
        
        energy_supplied = min(pv_production, consumption)
        total_energy_supplied_pv += energy_supplied

    if total_energy_demand > 0:
        ssr = (total_energy_supplied_pv / total_energy_demand) * 100
    else:
        ssr = 0
    ssr_values.append(ssr)

# Now plot
fig_ssr = go.Figure()

fig_ssr.add_trace(go.Scatter(
    x=pv_penetration_ratio,
    y=ssr_values,
    mode='lines+markers',
    name='SSR (%)',
    line=dict(color='green'),
    marker=dict(size=8),
    hovertemplate='<b>PV Penetration Ratio:</b> %{x:.4f}<br>'
                  '<b>SSR (%):</b> %{y:.2f}<br>'
                  '<b>Number of Panels:</b> %{customdata:,}',
    customdata=num_panels_range
))

fig_ssr.update_layout(
    title='Ense3 building - Self-Sufficiency Rate (SSR) vs. PV Penetration Ratio',
    xaxis_title='PV Penetration Ratio (Installed PV Capacity / Average Demand)',
    yaxis_title='SSR (Total Energy Supplied by PV / Total Energy Demand)',
    template='plotly_white',
    width=800,
    height=600,
    font=dict(size=12)
)

fig_ssr.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# ---------------------------
# 1) Basic System/Load Info
# ---------------------------
battery_capacity_per_unit = 57  # kWh per single 'battery unit' (if relevant)
converter_efficiency = 0.9      # 90%
initial_soc = 50               # 50% initial State of Charge

# Calculate total annual demand (kWh) and average demand (kW)
total_energy_demand = merged_data['Consumption_kWh'].sum()  # kWh over the entire dataset
num_hours = len(merged_data)  # e.g., 8760 for one year of hourly data
average_demand = total_energy_demand / num_hours  # [kW]

print(f"Total energy demand = {total_energy_demand:.1f} kWh")
print(f"Average demand = {average_demand:.2f} kW")

# ---------------------------
# 2) Define Ranges in Normalized Terms
# ---------------------------
# For example, let normalized PV capacity vary from 0.0 to 8.0
# => means up to 3 × average demand
norm_pv_range = np.arange(0.0, 8.1, 0.5)

# For battery, define hours of storage from 0 to 10
battery_hours_range = np.arange(0.0, 10.1, 2.0)

# Create 2D meshgrids for these two ranges
norm_pv_grid, battery_hours_grid = np.meshgrid(norm_pv_range, battery_hours_range)

# Prepare empty 2D array for SSR results
ssr_grid = np.zeros_like(norm_pv_grid, dtype=float)

# ---------------------------
# 3) Calculate SSR for Each Combination
# ---------------------------
for i in range(norm_pv_grid.shape[0]):
    for j in range(norm_pv_grid.shape[1]):
        
        # Extract the current normalized PV ratio & battery hours
        norm_pv = norm_pv_grid[i, j]        # dimensionless
        battery_hours = battery_hours_grid[i, j]  # hours

        # Convert normalized ratio -> installed PV power (kW)
        installed_pv_kW = norm_pv * average_demand

        # Convert hours -> battery capacity in kWh
        battery_capacity = battery_hours * average_demand

        # Track total energy that meets consumption from PV & battery
        total_energy_supplied_pv_to_demand = 0.0
        total_energy_discharged = 0.0

        # Initialize battery state-of-charge (SOC) in [%]
        soc = initial_soc

        # Go hour-by-hour (or time-step by time-step) in merged_data
        for _, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']  # kWh demanded in this hour
            # If 'PV_Production_kWh' is generation from a 1 kW reference system,
            # then the actual generation = (production_1kW * installed_pv_kW).
            pv_production = row['PV_Production_kWh'] * installed_pv_kW
            
            # 1) Direct supply
            direct_supply = min(pv_production, consumption)
            total_energy_supplied_pv_to_demand += direct_supply

            # 2) Battery interactions
            excess = pv_production - direct_supply   # leftover PV
            deficit = consumption - direct_supply    # unmet demand

            # Charge battery if we have excess PV
            if excess > 0 and battery_capacity > 0 and soc < 100:
                charge_potential = min(excess * converter_efficiency,
                                       (100 - soc)/100.0 * battery_capacity)
                soc += (charge_potential / battery_capacity) * 100.0
            
            # Discharge battery if we still have a deficit
            elif deficit > 0 and battery_capacity > 0 and soc > 20:
                discharge_potential = min(deficit,
                                          (soc - 20)/100.0 * battery_capacity * converter_efficiency)
                total_energy_discharged += discharge_potential
                soc -= (discharge_potential / battery_capacity) * 100.0

        # Compute SSR = fraction of total demand supplied by PV+Battery
        if total_energy_demand > 0:
            ssr = ((total_energy_supplied_pv_to_demand + total_energy_discharged)
                   / total_energy_demand) * 100.0
        else:
            ssr = 0.0
        
        # Store in the grid
        ssr_grid[i, j] = ssr

# ---------------------------
# 4) Plot a 3D Surface
# ---------------------------
fig = go.Figure(data=[
    go.Surface(
        x=norm_pv_grid,              # x-axis: normalized PV capacity
        y=battery_hours_grid,        # y-axis: battery hours
        z=ssr_grid,                  # z-axis: SSR (%)
        colorscale='Viridis',
        colorbar=dict(title='SSR (%)')
    )
])

fig.update_layout(
    title='Self-Sufficiency Rate vs. Normalized PV Capacity & Battery Hours',
    scene=dict(
        xaxis_title='PV Capacity / Average Demand',
        yaxis_title='Battery Utilize Capacity / ',
        zaxis_title='SSR (%)'
    ),
    template='plotly_white',
    width=800,
    height=600
)

fig.show()


Total energy demand = 1224408.3 kWh
Average demand = 139.39 kW


In [14]:
# Financial Parameters
panel_cost = 1220  # € per panel
battery_cost = 13000  # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7%
om_cost_batteries_rate = 0.05  # 5%

# System Parameters
battery_capacity_per_unit = 57  # kWh per battery
converter_efficiency = 0.9  # 90% efficiency
initial_soc = 50  # 50% initial battery SOC

# Number of Batteries (Fixed for Analysis)
num_batteries = 50

# Energy Demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# Define range of number of panels to analyze
panel_range = np.arange(0, 1501, 100)  

# Initialize list to store ROI values
roi_values = []

# Loop through each panel configuration
for num_panels in panel_range:
    # Calculate annual PV production
    annual_pv_production = merged_data['PV_Production_kWh'].sum() * num_panels 
    
    # Initialize annual metrics
    annual_energy_supplied_pv_to_demand = 0
    annual_energy_discharged = 0
    annual_excess_energy = 0
    annual_soc = initial_soc  # Reset SOC for the year
    
    battery_capacity = battery_capacity_per_unit * num_batteries
    
    for timestamp, row in merged_data.iterrows():
        consumption = row['Consumption_kWh']
        pv_production = row['PV_Production_kWh'] * num_panels 
        # Energy supplied to demand
        energy_supplied = min(pv_production, consumption)
        annual_energy_supplied_pv_to_demand += energy_supplied

        # Excess and deficit handling
        excess = pv_production - energy_supplied
        deficit = consumption - energy_supplied
        
        if excess > 0 and num_batteries > 0 and annual_soc < 100:
            charge = min(excess * converter_efficiency, (100 - annual_soc) / 100 * battery_capacity)
            annual_soc += (charge / battery_capacity) * 100
            remaining_excess = excess - (charge / converter_efficiency)
            annual_excess_energy += max(remaining_excess, 0)
        
        if deficit > 0 and num_batteries > 0 and annual_soc > 20:
            discharge = min(deficit, (annual_soc - 20) / 100 * battery_capacity * converter_efficiency)
            annual_energy_discharged += discharge
            annual_soc -= (discharge / battery_capacity) * 100

    # Calculate Financial Metrics
    total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
    
    om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
    om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
    total_om_cost = om_cost_panels + om_cost_batteries
    
    total_savings = annual_energy_supplied_pv_to_demand * energy_purchased_price
    revenue_surplus = annual_excess_energy * energy_sold_price
    
    net_savings = (total_savings + revenue_surplus) - total_om_cost
    
    # Calculate ROI
    roi = (net_savings / total_initial_investment) * 100 if total_initial_investment > 0 else 0
    roi_values.append(roi)

# Plot ROI vs. Number of Panels
fig_roi = go.Figure()

fig_roi.add_trace(go.Scatter(
    x=panel_range,
    y=roi_values,
    mode='lines+markers',
    name='ROI (%)',
    line=dict(color='blue'),
    marker=dict(size=8)
))

fig_roi.update_layout(
    title='Return on Investment (ROI) vs. Number of Panels',
    xaxis_title='Number of Panels',
    yaxis_title='ROI (%)',
    template='plotly_white',
    width=800,
    height=600
)

fig_roi.show()


In [15]:
import plotly.graph_objects as go
import numpy as np

# Financial Parameters
panel_cost = 1220  # € per panel
battery_cost = 13000  # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7%
om_cost_batteries_rate = 0.05  # 5%

# System Parameters
battery_capacity_per_unit = 57  # kWh per battery
converter_efficiency = 0.9  # 90% efficiency
initial_soc = 50  # 50% initial battery SOC

# Energy Demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# Define range of number of panels and batteries to analyze
panel_range = np.arange(500, 1501, 100)  # From 500 to 1500 panels in steps of 100
battery_range = np.arange(0, 101, 10)  # From 0 to 100 batteries in steps of 10

# Create a meshgrid for panels and batteries
panel_grid, battery_grid = np.meshgrid(panel_range, battery_range)
roi_grid = np.zeros_like(panel_grid, dtype=float)

# Loop through each combination of panels and batteries
for i in range(panel_grid.shape[0]):
    for j in range(panel_grid.shape[1]):
        num_panels = panel_grid[i, j]
        num_batteries = battery_grid[i, j]
        
        # Calculate annual PV production
        annual_pv_production = merged_data['PV_Production_kWh'].sum() * num_panels 
        
        # Initialize annual metrics
        annual_energy_supplied_pv_to_demand = 0
        annual_energy_discharged = 0
        annual_excess_energy = 0
        annual_soc = initial_soc  # Reset SOC for the year
        
        battery_capacity = battery_capacity_per_unit * num_batteries
        
        for timestamp, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']
            pv_production = row['PV_Production_kWh'] * num_panels 

            # Energy supplied to demand
            energy_supplied = min(pv_production, consumption)
            annual_energy_supplied_pv_to_demand += energy_supplied

            # Excess and deficit handling
            excess = pv_production - energy_supplied
            deficit = consumption - energy_supplied
            
            if excess > 0 and num_batteries > 0 and annual_soc < 100:
                charge = min(excess * converter_efficiency, (100 - annual_soc) / 100 * battery_capacity)
                annual_soc += (charge / battery_capacity) * 100
                remaining_excess = excess - (charge / converter_efficiency)
                annual_excess_energy += max(remaining_excess, 0)
            
            if deficit > 0 and num_batteries > 0 and annual_soc > 20:
                discharge = min(deficit, (annual_soc - 20) / 100 * battery_capacity * converter_efficiency)
                annual_energy_discharged += discharge
                annual_soc -= (discharge / battery_capacity) * 100

        # Calculate Financial Metrics
        total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
        
        om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
        om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
        total_om_cost = om_cost_panels + om_cost_batteries
        
        total_savings = annual_energy_supplied_pv_to_demand * energy_purchased_price
        revenue_surplus = annual_excess_energy * energy_sold_price
        
        net_savings = (total_savings + revenue_surplus) - total_om_cost
        
        # Calculate ROI
        roi = (net_savings / total_initial_investment) * 100 if total_initial_investment > 0 else 0
        roi_grid[i, j] = roi

# Create a 3D Surface Plot
fig = go.Figure(data=[go.Surface(
    z=roi_grid,
    x=panel_grid,
    y=battery_grid,
    colorscale='Viridis',
    colorbar=dict(title="ROI (%)")
)])

fig.update_layout(
    title='ROI vs. Number of Panels and Batteries',
    scene=dict(
        xaxis_title='Number of Panels',
        yaxis_title='Number of Batteries',
        zaxis_title='ROI (%)'
    ),
    template='plotly_white',
    width=800,
    height=600
)

fig.show()


In [16]:
import plotly.graph_objects as go
import numpy as np

# Financial Parameters
panel_cost = 1220  # € per panel
battery_cost = 13000  # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7%
om_cost_batteries_rate = 0.05  # 5%

# System Parameters
battery_capacity_per_unit = 57  # kWh per battery
converter_efficiency = 0.9  # 90% efficiency
initial_soc = 50  # 50% initial battery SOC

# Energy Demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# Define range of number of panels and batteries to analyze
panel_range = np.arange(500, 1501, 100)  # From 500 to 1500 panels in steps of 100
battery_range = np.arange(0, 101, 10)  # From 0 to 100 batteries in steps of 10

# Create a meshgrid for panels and batteries
panel_grid, battery_grid = np.meshgrid(panel_range, battery_range)
roi_grid = np.zeros_like(panel_grid, dtype=float)

# Store positive ROI scenarios
positive_roi_scenarios = []

# Loop through each combination of panels and batteries
for i in range(panel_grid.shape[0]):
    for j in range(panel_grid.shape[1]):
        num_panels = panel_grid[i, j]
        num_batteries = battery_grid[i, j]
        
        # Calculate annual PV production
        annual_pv_production = merged_data['PV_Production_kWh'].sum() * num_panels 
        
        # Initialize annual metrics
        annual_energy_supplied_pv_to_demand = 0
        annual_energy_discharged = 0
        annual_excess_energy = 0
        annual_soc = initial_soc  # Reset SOC for the year
        
        battery_capacity = battery_capacity_per_unit * num_batteries
        
        for timestamp, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']
            pv_production = row['PV_Production_kWh'] * num_panels 

            # Energy supplied to demand
            energy_supplied = min(pv_production, consumption)
            annual_energy_supplied_pv_to_demand += energy_supplied

            # Excess and deficit handling
            excess = pv_production - energy_supplied
            deficit = consumption - energy_supplied
            
            if excess > 0 and num_batteries > 0 and annual_soc < 100:
                charge = min(excess * converter_efficiency, (100 - annual_soc) / 100 * battery_capacity)
                annual_soc += (charge / battery_capacity) * 100
                remaining_excess = excess - (charge / converter_efficiency)
                annual_excess_energy += max(remaining_excess, 0)
            
            if deficit > 0 and num_batteries > 0 and annual_soc > 20:
                discharge = min(deficit, (annual_soc - 20) / 100 * battery_capacity * converter_efficiency)
                annual_energy_discharged += discharge
                annual_soc -= (discharge / battery_capacity) * 100

        # Calculate Financial Metrics
        total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
        
        om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
        om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
        total_om_cost = om_cost_panels + om_cost_batteries
        
        total_savings = annual_energy_supplied_pv_to_demand * energy_purchased_price
        revenue_surplus = annual_excess_energy * energy_sold_price
        
        net_savings = (total_savings + revenue_surplus) - total_om_cost
        
        # Calculate ROI
        roi = (net_savings / total_initial_investment) * 100 if total_initial_investment > 0 else 0
        roi_grid[i, j] = roi
        
        # Store positive ROI scenarios
        if roi > 0:
            positive_roi_scenarios.append((num_panels, num_batteries, roi))

# Print Positive ROI Scenarios
print("✅ **Positive ROI Scenarios:**")
for scenario in positive_roi_scenarios:
    print(f"Number of Panels: {scenario[0]}, Number of Batteries: {scenario[1]}, ROI: {scenario[2]:.2f}%")

# Create a 3D Surface Plot
fig = go.Figure()

# Surface plot for ROI
fig.add_trace(go.Surface(
    z=roi_grid,
    x=panel_grid,
    y=battery_grid,
    colorscale='Viridis',
    colorbar=dict(title="ROI (%)")
))

# Highlight Positive ROI Points
if positive_roi_scenarios:
    pos_x = [scenario[0] for scenario in positive_roi_scenarios]
    pos_y = [scenario[1] for scenario in positive_roi_scenarios]
    pos_z = [scenario[2] for scenario in positive_roi_scenarios]
    
    fig.add_trace(go.Scatter3d(
        x=pos_x,
        y=pos_y,
        z=pos_z,
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            symbol='circle',
            opacity=0.8
        ),
        name='Positive ROI'
    ))

# Update Layout
fig.update_layout(
    title='ROI vs. Number of Panels and Batteries with Positive ROI Highlighted',
    scene=dict(
        xaxis_title='Number of Panels',
        yaxis_title='Number of Batteries',
        zaxis_title='ROI (%)'
    ),
    template='plotly_white',
    width=800,
    height=600
)

fig.show()


✅ **Positive ROI Scenarios:**
Number of Panels: 500, Number of Batteries: 0, ROI: 8.95%
Number of Panels: 600, Number of Batteries: 0, ROI: 7.88%
Number of Panels: 700, Number of Batteries: 0, ROI: 6.76%
Number of Panels: 800, Number of Batteries: 0, ROI: 5.69%
Number of Panels: 900, Number of Batteries: 0, ROI: 4.75%
Number of Panels: 1000, Number of Batteries: 0, ROI: 3.92%
Number of Panels: 1100, Number of Batteries: 0, ROI: 3.20%
Number of Panels: 1200, Number of Batteries: 0, ROI: 2.56%
Number of Panels: 1300, Number of Batteries: 0, ROI: 2.01%
Number of Panels: 1400, Number of Batteries: 0, ROI: 1.51%
Number of Panels: 1500, Number of Batteries: 0, ROI: 1.06%
Number of Panels: 500, Number of Batteries: 10, ROI: 6.61%
Number of Panels: 600, Number of Batteries: 10, ROI: 6.10%
Number of Panels: 700, Number of Batteries: 10, ROI: 5.41%
Number of Panels: 800, Number of Batteries: 10, ROI: 4.67%
Number of Panels: 900, Number of Batteries: 10, ROI: 4.01%
Number of Panels: 1000, Number 

In [17]:
import plotly.graph_objects as go
import numpy as np

# Financial Parameters
panel_cost = 1220  # € per panel
battery_cost = 13000  # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7%
om_cost_batteries_rate = 0.05  # 5%

# System Parameters
battery_capacity_per_unit = 57  # kWh per battery
converter_efficiency = 0.9  # 90% efficiency
initial_soc = 50  # 50% initial battery SOC

# Energy Demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# Define range of panels and batteries
panel_range = np.arange(500, 1501, 100)  # From 500 to 1500 panels in steps of 100
battery_range = np.arange(0, 101, 10)  # From 0 to 100 batteries in steps of 10

# Create meshgrid for analysis
panel_grid, battery_grid = np.meshgrid(panel_range, battery_range)
roi_grid = np.zeros_like(panel_grid, dtype=float)

# Hover text grid
hover_texts = np.empty(panel_grid.shape, dtype=object)

# Store positive ROI scenarios
positive_roi_scenarios = []

# Loop through combinations
for i in range(panel_grid.shape[0]):
    for j in range(panel_grid.shape[1]):
        num_panels = panel_grid[i, j]
        num_batteries = battery_grid[i, j]
        
        battery_capacity = battery_capacity_per_unit * num_batteries
        total_pv_production = merged_data['PV_Production_kWh'].sum() * num_panels 
        
        # Initialize SOC and metrics
        soc = initial_soc
        total_energy_supplied_pv_to_demand = 0
        total_energy_discharged = 0
        annual_excess_energy = 0
        
        for timestamp, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']
            pv_production = row['PV_Production_kWh'] * num_panels 
            
            # Energy supplied
            energy_supplied_pv = min(pv_production, consumption)
            total_energy_supplied_pv_to_demand += energy_supplied_pv
            
            excess = pv_production - energy_supplied_pv
            deficit = consumption - energy_supplied_pv
            
            # Charge Battery
            if excess > 0 and num_batteries > 0 and soc < 100:
                charge = min(excess * converter_efficiency, (100 - soc) / 100 * battery_capacity)
                soc += (charge / battery_capacity) * 100
                annual_excess_energy += excess - charge / converter_efficiency
            
            # Discharge Battery
            if deficit > 0 and num_batteries > 0 and soc > 20:
                discharge = min(deficit, (soc - 20) / 100 * battery_capacity * converter_efficiency)
                total_energy_discharged += discharge
                soc -= (discharge / battery_capacity) * 100
        
        # Financial Metrics
        total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
        om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
        om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
        total_om_cost = om_cost_panels + om_cost_batteries
        
        total_savings = total_energy_supplied_pv_to_demand * energy_purchased_price
        revenue_surplus = annual_excess_energy * energy_sold_price
        net_savings = (total_savings + revenue_surplus) - total_om_cost
        
        # Calculate ROI
        roi = (net_savings / total_initial_investment) * 100 if total_initial_investment > 0 else 0
        roi_grid[i, j] = roi
        
        # Store positive ROI scenarios
        if roi > 0:
            positive_roi_scenarios.append((num_panels, num_batteries, roi))
        
        # Add custom hover text
        hover_texts[i, j] = (
            f"Number of Panels: {num_panels}<br>"
            f"Number of Batteries: {num_batteries}<br>"
            f"ROI: {roi:.2f}%"
        )

# Print Positive ROI Scenarios
print("✅ **Positive ROI Scenarios:**")
for scenario in positive_roi_scenarios:
    print(f"Number of Panels: {scenario[0]}, Number of Batteries: {scenario[1]}, ROI: {scenario[2]:.2f}%")

# Create a 3D Surface Plot
fig = go.Figure()

# Surface plot for ROI
fig.add_trace(go.Surface(
    z=roi_grid,
    x=panel_grid,
    y=battery_grid,
    colorscale='Viridis',
    colorbar=dict(title="ROI (%)"),
    hoverinfo='text',
    text=hover_texts
))

# Highlight Positive ROI Points
if positive_roi_scenarios:
    pos_x = [scenario[0] for scenario in positive_roi_scenarios]
    pos_y = [scenario[1] for scenario in positive_roi_scenarios]
    pos_z = [scenario[2] for scenario in positive_roi_scenarios]
    
    fig.add_trace(go.Scatter3d(
        x=pos_x,
        y=pos_y,
        z=pos_z,
        mode='markers+text',
        marker=dict(
            size=5,
            color='red',
            symbol='circle',
            opacity=0.8
        ),
        #text=hover_texts,
        #textposition='top center',
        #name='Positive ROI'
    ))

# Update Layout
fig.update_layout(
    title='ROI vs. Number of Panels and Batteries with Positive ROI Highlighted',
    scene=dict(
        xaxis_title='Number of Panels',
        yaxis_title='Number of Batteries',
        zaxis_title='ROI (%)'
    ),
    template='plotly_white',
    width=800,
    height=600
)

fig.show()


✅ **Positive ROI Scenarios:**
Number of Panels: 500, Number of Batteries: 0, ROI: 8.95%
Number of Panels: 600, Number of Batteries: 0, ROI: 7.88%
Number of Panels: 700, Number of Batteries: 0, ROI: 6.76%
Number of Panels: 800, Number of Batteries: 0, ROI: 5.69%
Number of Panels: 900, Number of Batteries: 0, ROI: 4.75%
Number of Panels: 1000, Number of Batteries: 0, ROI: 3.92%
Number of Panels: 1100, Number of Batteries: 0, ROI: 3.20%
Number of Panels: 1200, Number of Batteries: 0, ROI: 2.56%
Number of Panels: 1300, Number of Batteries: 0, ROI: 2.01%
Number of Panels: 1400, Number of Batteries: 0, ROI: 1.51%
Number of Panels: 1500, Number of Batteries: 0, ROI: 1.06%
Number of Panels: 500, Number of Batteries: 10, ROI: 6.61%
Number of Panels: 600, Number of Batteries: 10, ROI: 6.10%
Number of Panels: 700, Number of Batteries: 10, ROI: 5.41%
Number of Panels: 800, Number of Batteries: 10, ROI: 4.67%
Number of Panels: 900, Number of Batteries: 10, ROI: 4.01%
Number of Panels: 1000, Number 

In [19]:
import plotly.graph_objects as go
import numpy as np

# Financial Parameters
panel_cost = 1220  # € per panel
battery_cost = 13000  # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7%
om_cost_batteries_rate = 0.05  # 5%

# System Parameters
battery_capacity_per_unit = 57  # kWh per battery
converter_efficiency = 0.9  # 90% efficiency
initial_soc = 50  # 50% initial battery SOC

# Energy Demand
total_energy_demand = merged_data['Consumption_kWh'].sum()

# Define ranges for panels and batteries
battery_range = np.arange(0, 101, 10)  # 0 to 100 batteries in steps of 10
panel_range = np.arange(500, 3001, 100)  # 500 to 3000 panels in steps of 100

# Create meshgrid for analysis
panel_grid, battery_grid = np.meshgrid(panel_range, battery_range)
ssr_grid = np.zeros_like(panel_grid, dtype=float)
roi_grid = np.zeros_like(panel_grid, dtype=float)

# Store optimal scenario
optimal_scenario = {'panels': 0, 'batteries': 0, 'ssr': 0, 'roi': 0}

# Prepare custom hover text
hover_texts = np.empty(panel_grid.shape, dtype=object)

# Loop through combinations
for i in range(panel_grid.shape[0]):
    for j in range(panel_grid.shape[1]):
        num_panels = panel_grid[i, j]
        num_batteries = battery_grid[i, j]
        
        battery_capacity = battery_capacity_per_unit * num_batteries
        total_pv_production = merged_data['PV_Production_kWh'].sum() * num_panels 
        
        # Initialize SOC and metrics
        soc = initial_soc
        total_energy_supplied_pv_to_demand = 0
        total_energy_discharged = 0
        annual_excess_energy = 0
        
        for timestamp, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']
            pv_production = row['PV_Production_kWh'] * num_panels 
            
            # Energy supplied
            energy_supplied_pv = min(pv_production, consumption)
            total_energy_supplied_pv_to_demand += energy_supplied_pv
            
            excess = pv_production - energy_supplied_pv
            deficit = consumption - energy_supplied_pv
            
            # Charge Battery
            if excess > 0 and num_batteries > 0 and soc < 100:
                charge = min(excess * converter_efficiency, (100 - soc) / 100 * battery_capacity)
                soc += (charge / battery_capacity) * 100
                annual_excess_energy += excess - charge / converter_efficiency
            
            # Discharge Battery
            if deficit > 0 and num_batteries > 0 and soc > 20:
                discharge = min(deficit, (soc - 20) / 100 * battery_capacity * converter_efficiency)
                total_energy_discharged += discharge
                soc -= (discharge / battery_capacity) * 100
        
        # Calculate SSR
        ssr = ((total_energy_supplied_pv_to_demand + total_energy_discharged) / total_energy_demand) * 100 if total_energy_demand > 0 else 0
        ssr_grid[i, j] = ssr
        
        # Financial Metrics
        total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
        om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
        om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
        total_om_cost = om_cost_panels + om_cost_batteries
        
        total_savings = total_energy_supplied_pv_to_demand * energy_purchased_price
        revenue_surplus = annual_excess_energy * energy_sold_price
        net_savings = (total_savings + revenue_surplus) - total_om_cost
        
        roi = (net_savings / total_initial_investment) * 100 if total_initial_investment > 0 else 0
        roi_grid[i, j] = roi
        
        # Store optimal condition
        if roi > 0 and ssr > optimal_scenario['ssr']:
            optimal_scenario.update({
                'panels': num_panels,
                'batteries': num_batteries,
                'ssr': ssr,
                'roi': roi
            })

        # Prepare hover text
        hover_texts[i, j] = (
            f"Number of Panels: {num_panels}<br>"
            f"Number of Batteries: {num_batteries}<br>"
            f"SSR: {ssr:.2f}%<br>"
            f"ROI: {roi:.2f}%"
        )

# Print Optimal Scenario
print("✅ **Optimal Scenario (Maximum SSR with Positive ROI):**")
print(f"Number of Panels: {optimal_scenario['panels']}")
print(f"Number of Batteries: {optimal_scenario['batteries']}")
print(f"SSR: {optimal_scenario['ssr']:.2f}%")
print(f"ROI: {optimal_scenario['roi']:.2f}%")

# Plot SSR with Custom Hover
fig = go.Figure()

fig.add_trace(go.Surface(
    z=ssr_grid,
    x=panel_grid,
    y=battery_grid,
    surfacecolor=ssr_grid,
    colorscale='Viridis',
    colorbar=dict(title="SSR (%)"),
    hoverinfo='text',
    text=hover_texts
))

# Highlight Optimal Point
fig.add_trace(go.Scatter3d(
    x=[optimal_scenario['panels']],
    y=[optimal_scenario['batteries']],
    z=[optimal_scenario['ssr']],
    mode='markers+text',
    marker=dict(size=8, color='red'),
    text=['Optimal Point'],
    textposition='top center'
))

# Update Layout
fig.update_layout(
    title='Maximum SSR with Positive ROI',
    scene=dict(
        xaxis_title='Number of Panels',
        yaxis_title='Number of Batteries',
        zaxis_title='SSR (%)'
    ),
    template='plotly_white',
    width=800,
    height=600
)

fig.show()


✅ **Optimal Scenario (Maximum SSR with Positive ROI):**
Number of Panels: 1400
Number of Batteries: 40
SSR: 80.67%
ROI: 0.25%


In [22]:
import plotly.graph_objects as go
import numpy as np

# --------------------------------------------------
# 1) Financial & Technical Parameters
# --------------------------------------------------
panel_cost = 1220           # € per panel
battery_cost = 13000        # € per battery
energy_sold_price = 0.1208  # €/kWh
energy_purchased_price = 0.2713  # €/kWh
om_cost_panels_rate = 0.07  # 7% O&M cost of panel CAPEX
om_cost_batteries_rate = 0.05  # 5% O&M cost of battery CAPEX

battery_capacity_per_unit = 57  # kWh per "battery unit"
converter_efficiency = 0.9
initial_soc = 50          # %

# Panel capacity (kW) per physical panel
# For instance, if each panel is 610 W => 0.610 kW:
panel_capacity_kW = 0.610

# --------------------------------------------------
# 2) Load Data and Compute Averages
# --------------------------------------------------
# merged_data: DataFrame with columns ['Consumption_kWh', 'PV_Production_kWh'] at each time step
total_energy_demand = merged_data['Consumption_kWh'].sum()   # kWh
num_hours = len(merged_data)  # e.g. 8760 for 1 year of hourly data
average_demand = total_energy_demand / num_hours  # kW

print(f"Total annual demand = {total_energy_demand:.1f} kWh")
print(f"Average demand = {average_demand:.2f} kW")

# --------------------------------------------------
# 3) Define Dimensionless Ranges
# --------------------------------------------------
# Normalized PV ratio from 0.0 to 3.0 in steps of 0.5 => up to 3× average load
norm_pv_range = np.arange(0.0, 10.1, 0.5)

# Battery hours from 0 to 10 in steps of 2 => up to 10 hours of storage at average load
battery_hours_range = np.arange(0.0, 20.1, 2.0)

# Create meshgrids (X = norm_pv, Y = battery_hours)
norm_pv_grid, battery_hours_grid = np.meshgrid(norm_pv_range, battery_hours_range)

# Prepare 2D arrays for SSR and ROI
ssr_grid = np.zeros_like(norm_pv_grid, dtype=float)
roi_grid = np.zeros_like(norm_pv_grid, dtype=float)

# Store an "optimal scenario"
optimal_scenario = {
    'norm_pv': 0,
    'battery_hours': 0,
    'ssr': 0,
    'roi': 0
}

# For custom hover text
hover_texts = np.empty(norm_pv_grid.shape, dtype=object)

# --------------------------------------------------
# 4) Double Loop Over PV & Battery Hours
# --------------------------------------------------
for i in range(norm_pv_grid.shape[0]):
    for j in range(norm_pv_grid.shape[1]):
        
        # (A) Extract current dimensionless parameters
        norm_pv = norm_pv_grid[i, j]           # ratio
        battery_hours = battery_hours_grid[i, j]
        
        # (B) Convert to actual # of panels and # of batteries
        installed_pv_kW = norm_pv * average_demand
        num_panels = installed_pv_kW / panel_capacity_kW if panel_capacity_kW > 0 else 0
        
        battery_capacity_kWh = battery_hours * average_demand
        num_batteries = battery_capacity_kWh / battery_capacity_per_unit if battery_capacity_per_unit > 0 else 0
        
        # (C) Hourly simulation for SSR
        soc = initial_soc
        total_energy_supplied_pv_to_demand = 0.0
        total_energy_discharged = 0.0
        annual_excess_energy = 0.0
        
        for _, row in merged_data.iterrows():
            consumption = row['Consumption_kWh']
            # If 'PV_Production_kWh' is for a 1 kW reference system:
            pv_generation = row['PV_Production_kWh'] * installed_pv_kW
            
            # 1) Direct supply
            direct_supply = min(pv_generation, consumption)
            total_energy_supplied_pv_to_demand += direct_supply
            
            # 2) Battery logic
            excess = pv_generation - direct_supply
            deficit = consumption - direct_supply
            
            # Charge if we have excess
            if excess > 0 and num_batteries > 0 and soc < 100:
                # max we can store in this hour
                charge_max = (100 - soc)/100.0 * battery_capacity_kWh
                actual_charge = min(excess * converter_efficiency, charge_max)
                
                soc += (actual_charge / battery_capacity_kWh) * 100.0
                # The leftover after charging is "excess" that goes to the grid
                # That leftover = (excess - actual_charge / converter_efficiency)
                annual_excess_energy += max(0.0, excess - (actual_charge / converter_efficiency))
            
            # Discharge if there's deficit
            if deficit > 0 and num_batteries > 0 and soc > 20:
                # max we can discharge while respecting SoC floor of 20%
                discharge_max = (soc - 20)/100.0 * battery_capacity_kWh * converter_efficiency
                actual_discharge = min(deficit, discharge_max)
                
                total_energy_discharged += actual_discharge
                soc -= (actual_discharge / (battery_capacity_kWh * converter_efficiency)) * 100.0
        
        # (D) Calculate SSR
        ssr = 0.0
        if total_energy_demand > 0:
            ssr = ((total_energy_supplied_pv_to_demand + total_energy_discharged)
                   / total_energy_demand) * 100.0
        
        ssr_grid[i, j] = ssr
        
        # (E) Financial Calculations
        # CAPEX
        total_initial_investment = (panel_cost * num_panels) + (battery_cost * num_batteries)
        
        # O&M cost
        om_cost_panels = panel_cost * num_panels * om_cost_panels_rate
        om_cost_batteries = battery_cost * num_batteries * om_cost_batteries_rate
        total_om_cost = om_cost_panels + om_cost_batteries
        
        # Savings: any kWh not purchased from the grid => value at energy_purchased_price
        # Also, any surplus can be sold => revenue_surplus
        total_savings = total_energy_supplied_pv_to_demand * energy_purchased_price
        revenue_surplus = annual_excess_energy * energy_sold_price
        
        net_savings = (total_savings + revenue_surplus) - total_om_cost
        
        if total_initial_investment > 0:
            roi = (net_savings / total_initial_investment) * 100.0
        else:
            roi = 0.0
        
        roi_grid[i, j] = roi
        
        # (F) Update "Optimal" scenario (if you want max SSR subject to ROI>0, for example)
        # You can customize this criterion—here we pick the highest SSR that still has ROI>0
        if roi > 0 and ssr > optimal_scenario['ssr']:
            optimal_scenario.update({
                'norm_pv': norm_pv,
                'battery_hours': battery_hours,
                'ssr': ssr,
                'roi': roi
            })
        
        # (G) Hover text
        hover_texts[i, j] = (
            f"<b>norm_PV</b>: {norm_pv:.2f}<br>"
            f"<b>battery_hours</b>: {battery_hours:.1f}<br>"
            f"PV Panels: {num_panels:.1f}<br>"
            f"Batteries: {num_batteries:.1f}<br>"
            f"SSR: {ssr:.1f}%<br>"
            f"ROI: {roi:.1f}%<br>"
        )

# --------------------------------------------------
# 5) Print the "Optimal" Scenario
# --------------------------------------------------
print("Optimal Scenario (Max SSR with ROI>0):")
print(f"  Normalized PV = {optimal_scenario['norm_pv']:.2f} (i.e. {optimal_scenario['norm_pv']*100:.0f}% of average demand)")
print(f"  Battery Hours = {optimal_scenario['battery_hours']:.1f}")
print(f"  SSR = {optimal_scenario['ssr']:.2f}%")
print(f"  ROI = {optimal_scenario['roi']:.2f}%")

# --------------------------------------------------
# 6) Plot 3D Surface (SSR as Z-Axis)
# --------------------------------------------------
fig = go.Figure()

fig.add_trace(go.Surface(
    x=norm_pv_grid,              # X-axis: normalized PV
    y=battery_hours_grid,        # Y-axis: battery hours
    z=ssr_grid,                  # Z-axis: SSR
    surfacecolor=ssr_grid,       # for color
    colorscale='Viridis',
    colorbar=dict(title="SSR (%)"),
    hoverinfo='text',
    text=hover_texts
))

# Add a marker for the Optimal Scenario
fig.add_trace(go.Scatter3d(
    x=[optimal_scenario['norm_pv']],
    y=[optimal_scenario['battery_hours']],
    z=[optimal_scenario['ssr']],
    mode='markers+text',
    marker=dict(size=6, color='red'),
    text=["Optimal"],
    textposition='top center'
))

fig.update_layout(
    title='3D Surface: SSR vs. Normalized PV & Battery Hours',
    scene=dict(
        xaxis_title='(Installed PV (kW)) / (Average Demand (kW))',
        yaxis_title='Battery Hours (kWh / Average Demand)',
        zaxis_title='SSR (%)'
    ),
    template='plotly_white',
    width=900,
    height=700
)

fig.show()


Total annual demand = 1224408.3 kWh
Average demand = 139.39 kW
Optimal Scenario (Max SSR with ROI>0):
  Normalized PV = 6.00 (i.e. 600% of average demand)
  Battery Hours = 6.0
  SSR = 50.18%
  ROI = 0.11%
